In [1]:
from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

In [2]:
path.ls()

(#23) [Path('/root/.fastai/data/ml-100k/u4.base'),Path('/root/.fastai/data/ml-100k/u4.test'),Path('/root/.fastai/data/ml-100k/u.data'),Path('/root/.fastai/data/ml-100k/allbut.pl'),Path('/root/.fastai/data/ml-100k/mku.sh'),Path('/root/.fastai/data/ml-100k/ua.test'),Path('/root/.fastai/data/ml-100k/ua.base'),Path('/root/.fastai/data/ml-100k/u.item'),Path('/root/.fastai/data/ml-100k/u2.test'),Path('/root/.fastai/data/ml-100k/u3.test')...]

In [3]:
ratings = pd.read_csv(path/'u.data', delimiter='\t',header=None,
                      names=['user', 'movie', 'rating', 'timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
movies= pd.read_csv(path/'u.item',  delimiter='|',encoding='latin-1',
                    usecols=(0,1),names=('movie', 'title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [5]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,186,302,3,891717742,L.A. Confidential (1997)
2,22,377,1,878887116,Heavyweights (1994)
3,244,51,2,880606923,Legends of the Fall (1994)
4,166,346,1,886397596,Jackie Brown (1997)


In [6]:
ratings.rating.unique()

array([3, 1, 2, 4, 5])

Cross entropy loss:
$$ H(y, \hat{y}) = \sum_i y_i \log \frac{1}{\hat{y}_i} = -\sum_i y_i \log \hat{y}_i $$

But the rating have 1 to 5 indexing like this will fail as tesnor 0<sup>th</sup>. 

In [16]:
ratings.rating = ratings.rating -1
ratings.rating.unique()

array([2, 0, 1, 3, 4])

In [17]:
dls = CollabDataLoaders.from_df(ratings, user_name='user', item_name='title', rating_name='rating', bs=64)
dls.show_batch()

,user,title,rating
0,200,Assassins (1995),3
1,707,M*A*S*H (1970),2
2,236,Legends of the Fall (1994),4
3,773,Young Guns II (1990),1
4,648,"Hunchback of Notre Dame, The (1996)",2
5,542,Apocalypse Now (1979),2
6,406,Restoration (1995),2
7,457,Twister (1996),3
8,498,Three Colors: Red (1994),3
9,406,Forrest Gump (1994),3


In [18]:
class CollabNNCEL(Module):
    def __init__(self, user_sz, movie_sz, n_act=100):
        self.user_factors = Embedding(*user_sz)
        self.movie_factors = Embedding(*movie_sz)

        self.layers = nn.Sequential(
            nn.Linear(user_sz[1] + movie_sz[1], n_act),
            nn.ReLU(),
            nn.Linear(n_act, 5),
            nn.Softmax(dim=1),
        )

    
    def forward(self, x):
        embds = self.user_factors(x[:,0]), self.movie_factors(x[:,1])

        x = self.layers(torch.cat(embds, dim=1))
        return x

In [10]:
embs = get_emb_sz(dls)
embs

[(944, 74), (1665, 102)]

In [19]:
model = CollabNNCEL(*embs)

In [20]:
X, Y = dls.one_batch()
X.shape, Y.shape

(torch.Size([64, 2]), torch.Size([64, 1]))

In [21]:
y_pred = model.forward(X)
y_pred.shape, y_pred[0].sum()

(torch.Size([64, 5]), tensor(1., grad_fn=<SumBackward0>))

In [22]:
Y.shape, Y.squeeze().shape, Y.shape[0]

(torch.Size([64, 1]), torch.Size([64]), 64)

In [23]:
arr = [i for i in range(64)]
y_pred[arr,Y.squeeze().long()].shape, y_pred[arr,Y.squeeze().long()].mean()

(torch.Size([64]), tensor(0.2001, grad_fn=<MeanBackward0>))

In [24]:
def loss_func(pred, target):
    arr = [i for i in range(target.shape[0])]
    return pred[arr,target.squeeze().long()].mean()

In [25]:
loss_func(y_pred, Y)

tensor(0.2001, grad_fn=<MeanBackward0>)

In [27]:
learn_CE = Learner(dls, model, loss_func=loss_func)
learn_CE.fit_one_cycle(5, 5e-3, wd=0.01)

epoch,train_loss,valid_loss,time
0,0.033423,0.040593,00:14
1,0.031726,0.040079,00:16
2,0.027559,0.035988,00:19
3,0.023572,0.036266,00:14
4,0.025872,0.035778,00:14
